In [1]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

# This line will hide code by default when the notebook is exported as HTML
# di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

display(HTML("<style>.container { width:100% !important; }</style>"))


import hail as hl
hl.init(tmp_dir='/net/scratch/people/plggosborcz')

Running on Apache Spark version 2.4.5
SparkUI available at http://p1810.prometheus:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.62-84fa81b9ea3d
LOGGING: writing to /net/archive/groups/plggneuromol/GTS-analysis/imdik-zekanowski-gts/hail-20210622-1439-0.2.62-84fa81b9ea3d.log


In [2]:
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
from hail.plot import show
from pprint import pprint
from bokeh.layouts import gridplot
hl.plot.output_notebook()

import numpy as np
import pandas as pd
from functools import reduce
from itertools import chain
import statistics as stat

Loading BokehJS ...

In [4]:
from bokeh.plotting import figure, show, output_notebook
output_notebook()

Loading BokehJS ...

In [5]:
import bokeh.models.mappers

### note - paths in this notebook are outdated

## CADD annotation

In [6]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/GTS-with-pheno.mt')

In [7]:
cd = hl.import_table('/net/archive/groups/plggneuromol/GTS-analysis/cadd/whole_genome_SNVs.tsv.gz', force_bgz = True, delimiter='\t', comment='#', 
                     types = {'f0':'tstr', 'f1':'tint', 'f2':'tstr', 'f3':'tstr', 'f4':'tfloat', 'f5':'tfloat'},
                     no_header = True)

2021-06-22 13:47:01 Hail: WARN: '/net/archive/groups/plggneuromol/GTS-analysis/cadd/whole_genome_SNVs.tsv.gz' refers to no files


FatalError: HailException: arguments refer to no files

Java stack trace:
is.hail.utils.HailException: arguments refer to no files
	at is.hail.utils.ErrorHandling$class.fatal(ErrorHandling.scala:11)
	at is.hail.utils.package$.fatal(package.scala:77)
	at is.hail.expr.ir.TextTableReader$.readMetadata(TextTableReader.scala:253)
	at is.hail.expr.ir.TextTableReader$.apply(TextTableReader.scala:306)
	at is.hail.expr.ir.TextTableReader$.fromJValue(TextTableReader.scala:313)
	at is.hail.expr.ir.TableReader$.fromJValue(TableIR.scala:105)
	at is.hail.expr.ir.IRParser$.table_ir_1(Parser.scala:1470)
	at is.hail.expr.ir.IRParser$$anonfun$table_ir$1.apply(Parser.scala:1446)
	at is.hail.expr.ir.IRParser$$anonfun$table_ir$1.apply(Parser.scala:1446)
	at is.hail.utils.StackSafe$More.advance(StackSafe.scala:64)
	at is.hail.utils.StackSafe$.run(StackSafe.scala:16)
	at is.hail.utils.StackSafe$StackFrame.run(StackSafe.scala:32)
	at is.hail.expr.ir.IRParser$$anonfun$parse_table_ir$1.apply(Parser.scala:1965)


In [ ]:
cd = cd.select(variants = hl.parse_variant('chr'+hl.str(cd.f0)+':'+hl.str(cd.f1)+':'+hl.str(cd.f2)+':'+hl.str(cd.f3), reference_genome='GRCh38'),
               cadd_score = cd.f5)

In [ ]:
cd = hl.read_table('/net/archive/groups/plggneuromol/GTS-analysis/data/cadd-all.mt')

In [ ]:
contigs = ['chr1',
 'chr2',
 'chr3',
 'chr4',
 'chr5',
 'chr6',
 'chr7',
 'chr8',
 'chr9',
 'chr10',
 'chr11',
 'chr12',
 'chr13',
 'chr14',
 'chr15',
 'chr16',
 'chr17',
 'chr18',
 'chr19',
 'chr20',
 'chr21',
 'chr22']

In [ ]:
for c in contigs:
    cd_chr = cd.filter(cd.variants.locus.contig == c)
    cd_chr = cd_chr.key_by(cd_chr.variants.locus,cd_chr.variants.alleles)
    cd_chr.write('/net/archive/groups/plggneuromol/GTS-analysis/data/cadd-per-chr/'+c+'_cadd.mt')
    
cd_chr = cd.filter(hl.array(contigs).contains(cd.variants.locus.contig), keep = False)
cd_chr = cd_chr.key_by(cd_chr.variants.locus,cd_chr.variants.alleles)
cd_chr.write('/net/archive/groups/plggneuromol/GTS-analysis/data/cadd-per-chr/other_chroms_cadd.mt')

In [ ]:
#do the same for mts
for c in contigs:
    mt_chr = mt.filter_rows(mt.locus.contig == c)
    cd = hl.read_table('/net/archive/groups/plggneuromol/GTS-analysis/data/cadd-per-chr/'+c+'_cadd.mt')
    
    mt_chr = mt_chr.annotate_rows(cadd = cd[mt_chr.row_key])
    mt_chr.write('/net/archive/groups/plggneuromol/GTS-analysis/data/mt-annotated-per-chr/'+c+'-anno.mt')

In [ ]:
cd = hl.read_table('/net/archive/groups/plggneuromol/GTS-analysis/data/cadd-per-chr/other_chroms_cadd.mt')

mt_chr = mt.filter_rows(hl.array(contigs).contains(mt.locus.contig), keep = False)

mt_chr = mt_chr.annotate_rows(cadd =cd[mt_chr.row_key])

mt_chr.write('/net/archive/groups/plggneuromol/GTS-analysis/data/mt-annotated-per-chr/other_chroms-anno.mt')

In [ ]:
#now read all the matrices and join
mts = !ls /net/archive/groups/plggneuromol/GTS-analysis/data/mt-annotated-per-chr

In [ ]:
mt_chr = []

for m in mts:
    mt_chr.append(hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/mt-annotated-per-chr/'+m)) 

In [ ]:
mt = hl.MatrixTable.union_rows(*mt_chr)

mt = mt.transmute_rows(cadd = mt.cadd.cadd_score)

In [ ]:
mt.write('/net/archive/groups/plggneuromol/GTS-analysis/data/GTS-pheno-cadd.mt')

In [ ]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/GTS-pheno-cadd.mt')

## join with gnomad

In [ ]:
gd = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/gnomad/gnomad-ready-to-join.ht')

In [ ]:
#adjust column schema:

gd = gd.annotate_cols(s = hl.str(hl.str(gd.col_idx)+'gnomad'),
                      phenotypes = hl.struct(family = hl.str('gnomad'),
                                             sex = hl.str('gnomad'),
                                             kinship = hl.str('gnomad'),
                                             disease = hl.str('gnomad'),
                                             phenotype = hl.str('gnomad'),
                                             add_pheno = hl.str('gnomad'),
                                             heavy_tics = hl.str('gnomad')))
                      

In [ ]:
gd = gd.key_cols_by(gd.s)

In [ ]:
gd = gd.drop(gd.col_idx)

In [ ]:
mt = mt.union_cols(gd, row_join_type='outer')

In [ ]:
#filter out any rows that contain only refs:

mt = mt.filter_rows(hl.agg.any(mt.GT.is_non_ref()))

In [ ]:
mt.write('/net/archive/groups/plggneuromol/GTS-analysis/data/GTS-gnomad.mt')

## annotate mt with nearest_genes_20kb



In [1]:
genes = hl.read_table('/net/archive/groups/plggneuromol/GTS-analysis/analysis/vcf_preprocessing/genecode_v32.ht')
genes = genes.filter(hl.is_valid_contig(genes['hg38.knownGene.chrom'], reference_genome='GRCh38'))


start = hl.cond(genes['hg38.knownGene.txStart'] < 20000, 1, genes['hg38.knownGene.txStart'] - 20000)
stop =  hl.cond(hl.contig_length(genes['hg38.knownGene.chrom'], reference_genome='GRCh38') - genes['hg38.knownGene.txEnd'] < 20000, 
                hl.contig_length(genes['hg38.knownGene.chrom'], reference_genome='GRCh38'),
                genes['hg38.knownGene.txEnd'] + 20000)

genes = genes.transmute(interval = 
                        hl.locus_interval(genes['hg38.knownGene.chrom'], 
                                          start,
                                          stop,
                                          reference_genome='GRCh38'))

genes = genes.key_by(genes.interval)

mt = mt.annotate_rows(nearest_genes_20kb = hl.array(hl.set(genes.index(mt.locus, all_matches=True)['hg38.kgXref.geneSymbol'])))

NameError: name 'hl' is not defined

## Make the simulated controls gender equal

PAR coordinates in GrCh38: http://m.ensembl.org/info/genome/genebuild/human_PARS.html

In [ ]:
par1 = [10001,2781479]
par2 = [155701383,156030895]

In [ ]:
rg = hl.get_reference('GRCh38')

In [ ]:
rg.lengths['chrX']

In [ ]:
nonpars = [[1,10000],[2781480,155701382]]

### first see sex distribution in our samples

In [ ]:
samples = mt.s.collect()

gnomad_females = samples[185:(185+70)] 

gnomads_males = samples[(185+70):]

mt = mt.transmute_cols(phenotypes = mt.phenotypes.annotate(sex = hl.cond(mt.s.contains('gnomad'),
                                                      hl.cond(hl.array(gnomad_females).contains(mt.s),
                                                            'F', 'M'),
                                                      mt.phenotypes['sex'])))

In [ ]:
mt.aggregate_cols(hl.agg.counter(mt.phenotypes['sex']))

In [ ]:
mt = mt.annotate_entries(GT = hl.cond(
                         mt.locus.contig == 'chrX',
                         hl.cond(
                            mt.phenotypes['sex'] == 'F',
                            mt.GT,
                            hl.cond(
                                mt.locus.position > nonpars[1][1], 
                                mt.GT,
                                hl.cond(
                                    mt.locus.position < nonpars[0][1],
                                    hl.cond(
                                        mt.locus.position > nonpars[0][0],
                                        mt.GT,
                                        hl.call(mt.GT[0], mt.GT[0])),
                                    hl.call(mt.GT[0], mt.GT[0])))),
                         mt.GT))     

In [ ]:
#mt.write('/net/archive/groups/plggneuromol/GTS-analysis/data/GTS-gnomad-sex.mt')

prepare matrix tables

In [ ]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/GTS-gnomad-sex.mt')

mt = mt.filter_rows(mt.cadd > 0) # this is so I don't do too many tests

mt_subset = mt.sample_rows(0.002)

mt.count()

mt_subset.count()

mt = mt.annotate_cols(category = hl.cond(mt.s.contains('gnomad'), False, (mt.phenotypes.disease == 'YES')))
mt_subset = mt_subset.annotate_cols(category = hl.cond(mt_subset.s.contains('gnomad'), False, (mt_subset.phenotypes.disease == 'YES')))

mt = mt.filter_cols((mt.s.contains('gnomad')) | ((mt.phenotypes.family =='.') & (mt.phenotypes.disease =='YES')))
mt_subset = mt_subset.filter_cols((mt_subset.s.contains('gnomad')) | ((mt_subset.phenotypes.family =='.') & (mt_subset.phenotypes.disease =='YES')))
samples = mt.s.collect()

#first 70 gnomad samples are females

to_keep = samples[0:44] + samples[110:144]
mt = mt.filter_cols(hl.array(to_keep).contains(mt.s)) # filter out excessive gnomads
mt_subset = mt_subset.filter_cols(hl.array(to_keep).contains(mt_subset.s))

mt_test = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/GTS-gnomad-sex.mt')
mt_test = mt_test.filter_rows(mt_test.cadd > 0) 
mt_test = mt_test.annotate_cols(category = hl.cond(mt_test.s.contains('gnomad'), False, (mt_test.phenotypes.disease == 'YES')))
mt_test = mt_test.filter_cols(mt_test.phenotypes.family == '.', keep = False)
mt_test = mt_test.filter_cols((mt_test.phenotypes.disease == 'YES') | (mt_test.phenotypes.disease == 'NO'))

In [ ]:
# generate new alternative data subsets:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/mt-for-skat.mt')


mt_subset_2 = mt.sample_rows(0.002)
mt_subset_2.write('/net/archive/groups/plggneuromol/GTS-analysis/data/mt-subset-2.mt')

In [7]:
mt_subset_3 = mt.sample_rows(0.002)
mt_subset_3.write('/net/archive/groups/plggneuromol/GTS-analysis/data/mt-subset-3.mt')

2020-10-29 11:20:54 Hail: INFO: wrote matrix table with 12721 rows and 78 columns in 6622 partitions to /net/archive/groups/plggneuromol/GTS-analysis/data/mt-subset-3.mt


In [ ]:
mt.write('/net/archive/groups/plggneuromol/GTS-analysis/data/mt-for-skat.mt')
mt_subset.write('/net/archive/groups/plggneuromol/GTS-analysis/data/mt-subset.mt')
mt_test.write('/net/archive/groups/plggneuromol/GTS-analysis/data/mt-test.mt')

### Perform PCA on subseted genotypes:

In [6]:
# this matrix table has 40 gnomads that we need for PCA, but lacks normal samples
mt_gnomads = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/mt-for-skat-may.mt')

In [7]:
#this matrix table has all the samples but too many gnomads
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/GTS-gnomad-sex.mt')

In [8]:
gnomads = mt_gnomads.s.collect()

In [9]:
gnomads = gnomads[40:]

In [10]:
samples = mt.s.collect()

In [11]:
samples = samples[0:185]

In [12]:
to_keep = gnomads + samples

In [13]:
mt = mt.filter_cols(hl.array(to_keep).contains(mt.s))

In [14]:
mt.count() 

(11825581, 225)

Remove excesive GnomADs (we only need 40, the same samples go into the SKAT test)

In [15]:
subset = mt.sample_rows(0.001)

In [16]:
subset.count()

(11940, 225)

In [17]:
subset.write('/net/archive/groups/plggneuromol/GTS-analysis/data/subset.mt')

2021-06-22 14:41:42 Hail: INFO: wrote matrix table with 11940 rows and 225 columns in 6622 partitions to /net/archive/groups/plggneuromol/GTS-analysis/data/subset.mt
    Total size: 14.14 MiB
    * Rows/entries: 14.14 MiB
    * Columns: 2.17 KiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  4 rows (21.93 KiB)


In [18]:
subset = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/subset.mt')

In [19]:
eigenvalues, pcs, _ = hl.hwe_normalized_pca(subset.GT)

2021-06-22 14:41:56 Hail: INFO: hwe_normalized_pca: running PCA using 9173 variants.
2021-06-22 14:42:09 Hail: INFO: pca: running PCA with 10 components...


In [20]:
mt = mt.annotate_cols(scores = pcs[mt.s].scores)

In [21]:
mt.write('/net/archive/groups/plggneuromol/GTS-analysis/data/pca.mt')

2021-06-22 15:12:47 Hail: INFO: wrote matrix table with 11825581 rows and 225 columns in 6622 partitions to /net/archive/groups/plggneuromol/GTS-analysis/data/pca.mt
    Total size: 9.03 GiB
    * Rows/entries: 9.03 GiB
    * Columns: 20.17 KiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  8022 rows (7.38 MiB)


In [22]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/pca.mt')

In [23]:
import bokeh.palettes
from bokeh.models import CategoricalColorMapper

colors = bokeh.palettes.plasma(26*8)

In [24]:
del colors[1::2] #run this 3times to remove every second color
del colors[1::2]
del colors[1::2]

In [25]:
len(colors)

26

In [48]:
p = hl.plot.scatter(mt.scores[8],
                    mt.scores[9],
                    label=(mt.phenotypes.family),
                    title='PCA', xlabel='PC9', ylabel='PC10',
                    colors = CategoricalColorMapper(palette=colors, factors=list(set(mt.phenotypes.family.collect()))))

p.xaxis.axis_label_text_font_size = "15pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.axis_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"

In [49]:
show(p)

In [28]:
PC1G = mt.filter_cols(mt.phenotypes.family == 'gnomad').scores[0].collect()

In [29]:
PC1 = mt.filter_cols(mt.phenotypes.family == 'gnomad', keep = False).scores[0].collect()

In [30]:
PC2G = mt.filter_cols(mt.phenotypes.family == 'gnomad').scores[1].collect()

In [31]:
PC2 = mt.filter_cols(mt.phenotypes.family == 'gnomad', keep = False).scores[1].collect()

In [32]:
max(PC2 + PC2G)/np.std(PC2 + PC2G)

4.931378437608183

In [33]:
for a in [PC1, PC2, PC1G, PC2G]:
    print(np.median(a))

-0.027545759201703376
-0.02300988992800349
0.013283901225657936
-0.01049703382304132


In [34]:
np.std(PC1 + PC1G)

0.11852730868027855

In [35]:
gts = np.column_stack((PC1, PC2))

In [36]:
gno = np.column_stack((PC1G, PC2G))

In [37]:
np.median(np.linalg.norm(gts, axis = 1))

0.055600152393757755

In [38]:
gno = np.column_stack((PC1G, PC2G))

In [39]:
np.median(np.linalg.norm(gno, axis = 1))

0.019651188641982106